Cargamos los datos en un data frame e importamos librerias

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import contextily as ctx

df=pd.read_csv('Airbnb_Open_Data.csv')

C:\Users\Nico\AppData\Local\Temp\ipykernel_15456\853144216.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('Airbnb_Open_Data.csv')


Vemos los primeros datos del Data Frame

In [2]:

df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [4]:
df.isnull().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified               289
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    214
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15893
reviews per month                  15879
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

Convertimosa a float price y service fee

In [5]:
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['service fee'] = df['service fee'].replace('[\$,]', '', regex=True).astype(float)

df.head(1)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Nico\AppData\Local\Temp\ipykernel_15456\4126109399.py:1: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\Nico\AppData\Local\Temp\ipykernel_15456\4126109399.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['service fee'] = df['service fee'].replace('[\$,]', '', regex=True).astype(float)


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,193.0,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN


In [6]:
df.describe()

,id,host id,lat,long,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
count,1.025990e+05,1.025990e+05,102591.000000,102591.000000,102385.000000,102352.000000,102326.000000,102190.000000,102416.000000,86720.000000,102273.000000,102280.000000,102151.000000
mean,2.914623e+07,4.925411e+10,40.728094,-73.949644,2012.487464,625.293536,125.026924,8.135845,27.483743,1.374022,3.279106,7.936605,141.133254
std,1.625751e+07,2.853900e+10,0.055857,0.049521,5.765556,331.671614,66.325739,30.553781,49.508954,1.746621,1.284657,32.218780,135.435024
min,1.001254e+06,1.236005e+08,40.499790,-74.249840,2003.000000,50.000000,10.000000,-1223.000000,0.000000,0.010000,1.000000,1.000000,-10.000000
25%,1.508581e+07,2.458333e+10,40.688740,-73.982580,2007.000000,340.000000,68.000000,2.000000,1.000000,0.220000,2.000000,1.000000,3.000000
50%,2.913660e+07,4.911774e+10,40.722290,-73.954440,2012.000000,624.000000,125.000000,3.000000,7.000000,0.740000,3.000000,1.000000,96.000000
75%,4.320120e+07,7.399650e+10,40.762760,-73.932350,2017.000000,913.000000,183.000000,5.000000,30.000000,2.000000,4.000000,2.000000,269.000000
max,5.736742e+07,9.876313e+10,40.916970,-73.705220,2022.000000,1200.000000,240.000000,5645.000000,1024.000000,90.000000,5.000000,332.000000,3677.000000


Podemos determinar que hay una gran cantidad de nulos en varias columnas. Ademas en las columnas availability 365 y minimun nights los valores minimos son negativos y los maximos estan muy por encima de lo esperado

In [7]:
filas_noches_minimas_negativas = df[df['minimum nights'] <= 0]
print(filas_noches_minimas_negativas.shape[0])
filas_noches_minimas_excedentes = df[df['minimum nights'] >365]
filas_noches_minimas_excedentes.shape[0]

13


35

al ser pocas filas las eliminamos

In [8]:
# Eliminar filas con valores anómalos en availability_365
df_limpio = df[(df['minimum nights'] >= 0) & (df['minimum nights'] <= 365)]


Ahora lo hacemos con la disponibilidad 365

In [9]:
filas_disponibilidad_negativas = df_limpio[df_limpio['availability 365'] <= 0]
print(filas_disponibilidad_negativas.shape[0])
filas_disponibilidad_excedentes = df_limpio[df_limpio['availability 365'] >365]
filas_disponibilidad_excedentes.shape[0]

23925


2765

Como son valores que representan una cantidad considerable del data set hay que adecuarlos y no eliminarlos. LOs valores superiores a 365 quedan en 365

In [10]:
# Convertir valores negativos a positivos en availability_365
df_limpio['availability 365'] = df_limpio['availability 365'].abs()

# Corregir valores mayores a 365 a 365
df_limpio['availability 365'] = df_limpio['availability 365'].apply(lambda x: 365 if x > 365 else x)

# Verificar los cambios
df_limpio.describe()

C:\Users\Nico\AppData\Local\Temp\ipykernel_15456\400343744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['availability 365'] = df_limpio['availability 365'].abs()
C:\Users\Nico\AppData\Local\Temp\ipykernel_15456\400343744.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['availability 365'] = df_limpio['availability 365'].apply(lambda x: 365 if x > 365 else x)


,id,host id,lat,long,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
count,1.021420e+05,1.021420e+05,102134.000000,102134.000000,101940.000000,101895.000000,101869.000000,102142.000000,101961.000000,86338.000000,101837.000000,101823.000000,101718.000000
mean,2.919016e+07,4.925833e+10,40.728089,-73.949623,2012.486747,625.322302,125.032463,7.856513,27.436490,1.373167,3.279869,7.934524,140.187115
std,1.624834e+07,2.853797e+10,0.055866,0.049526,5.764688,331.664484,66.324395,17.051180,49.393521,1.745033,1.284527,32.194386,133.423444
min,1.001254e+06,1.236005e+08,40.499790,-74.249840,2003.000000,50.000000,10.000000,1.000000,0.000000,0.010000,1.000000,1.000000,0.000000
25%,1.513870e+07,2.458329e+10,40.688733,-73.982560,2007.000000,340.000000,68.000000,2.000000,1.000000,0.220000,2.000000,1.000000,3.000000
50%,2.919266e+07,4.912144e+10,40.722290,-73.954440,2012.000000,625.000000,125.000000,3.000000,7.000000,0.740000,3.000000,1.000000,96.000000
75%,4.323944e+07,7.400619e+10,40.762770,-73.932330,2017.000000,913.000000,183.000000,5.000000,30.000000,2.000000,4.000000,2.000000,268.000000
max,5.736742e+07,9.876313e+10,40.916970,-73.705220,2022.000000,1200.000000,240.000000,365.000000,1024.000000,90.000000,5.000000,332.000000,365.000000


Eliminamos las columnas que primeramente no vamos a necesitar

In [11]:
df_limpio.columns

Index(['id', 'NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review', 'reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'license'],
      dtype='object')

In [12]:
columnas_a_eliminar=['id', 'NAME', 'host id', 'host_identity_verified', 'host name', 'country',
       'country code', 'last review', 'house_rules', 'license']

df_limpio=df_limpio.drop(columns=columnas_a_eliminar, axis=1)
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102142 entries, 0 to 102598
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   neighbourhood group             102114 non-null  object 
 1   neighbourhood                   102127 non-null  object 
 2   lat                             102134 non-null  float64
 3   long                            102134 non-null  float64
 4   instant_bookable                102046 non-null  object 
 5   cancellation_policy             102067 non-null  object 
 6   room type                       102142 non-null  object 
 7   Construction year               101940 non-null  float64
 8   price                           101895 non-null  float64
 9   service fee                     101869 non-null  float64
 10  minimum nights                  102142 non-null  float64
 11  number of reviews               101961 non-null  float64
 12  reviews per month    

In [13]:
df_limpio.head()

,neighbourhood group,neighbourhood,lat,long,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
0,Brooklyn,Kensington,40.64749,-73.97237,False,strict,Private room,2020.0,966.0,193.0,10.0,9.0,0.21,4.0,6.0,286.0
1,Manhattan,Midtown,40.75362,-73.98377,False,moderate,Entire home/apt,2007.0,142.0,28.0,30.0,45.0,0.38,4.0,2.0,228.0
2,Manhattan,Harlem,40.80902,-73.94190,True,flexible,Private room,2005.0,620.0,124.0,3.0,0.0,NaN,5.0,1.0,352.0
3,Brooklyn,Clinton Hill,40.68514,-73.95976,True,moderate,Entire home/apt,2005.0,368.0,74.0,30.0,270.0,4.64,4.0,1.0,322.0
4,Manhattan,East Harlem,40.79851,-73.94399,False,moderate,Entire home/apt,2009.0,204.0,41.0,10.0,9.0,0.10,3.0,1.0,289.0


In [14]:
df_limpio.isnull().sum()

neighbourhood group                  28
neighbourhood                        15
lat                                   8
long                                  8
instant_bookable                     96
cancellation_policy                  75
room type                             0
Construction year                   202
price                               247
service fee                         273
minimum nights                        0
number of reviews                   181
reviews per month                 15804
review rate number                  305
calculated host listings count      319
availability 365                    424
dtype: int64

Reemplazamos los valores nulos de las columnas

In [15]:
#Al ser pocos los eliminamos
df_limpio = df_limpio.dropna(subset=['neighbourhood group', 'neighbourhood'])
df_limpio = df_limpio.dropna(subset=['lat', 'long'])
#Reemplazamos por moda
df_limpio['instant_bookable'].fillna(df_limpio['instant_bookable'].mode().iloc[0], inplace=True)
df_limpio['cancellation_policy'].fillna(df_limpio['cancellation_policy'].mode().iloc[0], inplace=True)
#reemplazamos por mediana
median_año = df['Construction year'].median()
df_limpio['Construction year'].fillna(median_año, inplace=True)
median_noches = df['minimum nights'].median()
df_limpio['minimum nights'].fillna(median_noches, inplace=True)
median_reviews = df['number of reviews'].median()
df_limpio['number of reviews'].fillna(median_reviews, inplace=True)
median_mensual = df['reviews per month'].median()
df_limpio['reviews per month'].fillna(median_mensual, inplace=True)
median_puntaje = df['review rate number'].median()
df_limpio['review rate number'].fillna(median_puntaje, inplace=True)
median_host = df['calculated host listings count'].median()
df_limpio['calculated host listings count'].fillna(median_host, inplace=True)
median_availability = df['availability 365'].median()
df_limpio['availability 365'].fillna(median_availability, inplace=True)

median_precio = df['price'].median()
df_limpio['price'].fillna(median_precio, inplace=True)
median_tarifa = df['service fee'].median()
df_limpio['service fee'].fillna(median_tarifa, inplace=True)


df_limpio.isnull().sum()



neighbourhood group               0
neighbourhood                     0
lat                               0
long                              0
instant_bookable                  0
cancellation_policy               0
room type                         0
Construction year                 0
price                             0
service fee                       0
minimum nights                    0
number of reviews                 0
reviews per month                 0
review rate number                0
calculated host listings count    0
availability 365                  0
dtype: int64

Analizamos la cantidad de alquileres con noches minimas = 0

In [16]:
filas_disponibilidad_negativas = df_limpio[df_limpio['availability 365'] <= 0]
print(filas_disponibilidad_negativas.shape[0])

23498


Consideramos que estos alquileres al estar en 0 no estan disponibles por lo que agregamos una columna para indicar que no estan disponibles

In [17]:
# Crear una columna dummy para indicar si la propiedad está disponible
df_limpio['is_available'] = df_limpio['availability 365'] > 0

# Conversión de 'is_available' a numérico (0 o 1)
df_limpio['is_available'] = df_limpio['is_available'].astype(int)

In [18]:
df_limpio.head()

,neighbourhood group,neighbourhood,lat,long,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365,is_available
0,Brooklyn,Kensington,40.64749,-73.97237,False,strict,Private room,2020.0,966.0,193.0,10.0,9.0,0.21,4.0,6.0,286.0,1
1,Manhattan,Midtown,40.75362,-73.98377,False,moderate,Entire home/apt,2007.0,142.0,28.0,30.0,45.0,0.38,4.0,2.0,228.0,1
2,Manhattan,Harlem,40.80902,-73.94190,True,flexible,Private room,2005.0,620.0,124.0,3.0,0.0,0.74,5.0,1.0,352.0,1
3,Brooklyn,Clinton Hill,40.68514,-73.95976,True,moderate,Entire home/apt,2005.0,368.0,74.0,30.0,270.0,4.64,4.0,1.0,322.0,1
4,Manhattan,East Harlem,40.79851,-73.94399,False,moderate,Entire home/apt,2009.0,204.0,41.0,10.0,9.0,0.10,3.0,1.0,289.0,1


In [19]:
df_limpio.describe()

,lat,long,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365,is_available
count,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000
mean,40.728085,-73.949622,2012.485694,625.299987,125.028514,7.853738,27.377115,1.275430,3.279114,7.915115,139.993045,0.769833
std,0.055868,0.049530,5.759069,331.276188,66.238244,17.039993,49.319563,1.620872,1.282658,32.153950,133.186587,0.420942
min,40.499790,-74.249840,2003.000000,50.000000,10.000000,1.000000,0.000000,0.010000,1.000000,1.000000,0.000000,0.000000
25%,40.688730,-73.982560,2008.000000,341.000000,68.000000,2.000000,1.000000,0.280000,2.000000,1.000000,4.000000,1.000000
50%,40.722280,-73.954430,2012.000000,624.000000,125.000000,3.000000,7.000000,0.740000,3.000000,1.000000,96.000000,1.000000
75%,40.762770,-73.932320,2017.000000,912.000000,182.000000,5.000000,30.000000,1.710000,4.000000,2.000000,268.000000,1.000000
max,40.916970,-73.705220,2022.000000,1200.000000,240.000000,365.000000,1024.000000,90.000000,5.000000,332.000000,365.000000,1.000000


In [20]:
print("Nro filas antes:")
print(len(df))
print("Nro filas despues:")
print(len(df_limpio))

Nro filas antes:
102599
Nro filas despues:
102091


Analizamos los grupos de barrios

In [21]:
df_limpio.groupby(['neighbourhood group']).size()

neighbourhood group
Bronx             2703
Brooklyn         41651
Manhattan        43570
Queens           13214
Staten Island      951
brookln              1
manhatan             1
dtype: int64

Vemos que hay categorias repetidas, las unimos

In [22]:
df_limpio['neighbourhood group']=df_limpio['neighbourhood group'].replace('brookln','Brooklyn')
df_limpio['neighbourhood group']=df_limpio['neighbourhood group'].replace('manhatan','Manhattan')
df_limpio.groupby(['neighbourhood group']).size()

neighbourhood group
Bronx             2703
Brooklyn         41652
Manhattan        43571
Queens           13214
Staten Island      951
dtype: int64

Las variables categoricas neighbourhood group, neighbourhood, instant_bookable, cancellation_policy y room type se reemplazan por numeros

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
encoder = LabelEncoder()
for column in df_limpio.columns:
    if df_limpio[column].dtype == 'object' or df_limpio[column].dtype =='bool':
        df_limpio[column] = encoder.fit_transform(df_limpio[column])
        print(f"\nMapping of '{column}' categories to encoded values:")
        for original_category, encoded_value in zip(encoder.classes_, encoder.transform(encoder.classes_)):
            print(f"{original_category}: {encoded_value}")
            
df_limpio.head()


Mapping of 'neighbourhood group' categories to encoded values:
Bronx: 0
Brooklyn: 1
Manhattan: 2
Queens: 3
Staten Island: 4

Mapping of 'neighbourhood' categories to encoded values:
Allerton: 0
Arden Heights: 1
Arrochar: 2
Arverne: 3
Astoria: 4
Bath Beach: 5
Battery Park City: 6
Bay Ridge: 7
Bay Terrace: 8
Bay Terrace, Staten Island: 9
Baychester: 10
Bayside: 11
Bayswater: 12
Bedford-Stuyvesant: 13
Belle Harbor: 14
Bellerose: 15
Belmont: 16
Bensonhurst: 17
Bergen Beach: 18
Boerum Hill: 19
Borough Park: 20
Breezy Point: 21
Briarwood: 22
Brighton Beach: 23
Bronxdale: 24
Brooklyn Heights: 25
Brownsville: 26
Bull's Head: 27
Bushwick: 28
Cambria Heights: 29
Canarsie: 30
Carroll Gardens: 31
Castle Hill: 32
Castleton Corners: 33
Chelsea: 34
Chelsea, Staten Island: 35
Chinatown: 36
City Island: 37
Civic Center: 38
Claremont Village: 39
Clason Point: 40
Clifton: 41
Clinton Hill: 42
Co-op City: 43
Cobble Hill: 44
College Point: 45
Columbia St: 46
Concord: 47
Concourse: 48
Concourse Village: 49


,neighbourhood group,neighbourhood,lat,long,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365,is_available
0,1,111,40.64749,-73.97237,0,2,2,2020.0,966.0,193.0,10.0,9.0,0.21,4.0,6.0,286.0,1
1,2,130,40.75362,-73.98377,0,1,0,2007.0,142.0,28.0,30.0,45.0,0.38,4.0,2.0,228.0,1
2,2,97,40.80902,-73.94190,1,0,2,2005.0,620.0,124.0,3.0,0.0,0.74,5.0,1.0,352.0,1
3,1,42,40.68514,-73.95976,1,1,0,2005.0,368.0,74.0,30.0,270.0,4.64,4.0,1.0,322.0,1
4,2,62,40.79851,-73.94399,0,1,0,2009.0,204.0,41.0,10.0,9.0,0.10,3.0,1.0,289.0,1


Guardo el csv

In [25]:
df_limpio.to_csv('Airbnb_limpio_v2.csv', index=False)

In [26]:
df_limpio.describe()

,neighbourhood group,neighbourhood,lat,long,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365,is_available
count,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000,102091.000000
mean,1.687122,108.371551,40.728085,-73.949622,0.497341,1.000245,0.974023,2012.485694,625.299987,125.028514,7.853738,27.377115,1.275430,3.279114,7.915115,139.993045,0.769833
std,0.763349,69.523230,0.055868,0.049530,0.499995,0.815194,1.031254,5.759069,331.276188,66.238244,17.039993,49.319563,1.620872,1.282658,32.153950,133.186587,0.420942
min,0.000000,0.000000,40.499790,-74.249840,0.000000,0.000000,0.000000,2003.000000,50.000000,10.000000,1.000000,0.000000,0.010000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,52.000000,40.688730,-73.982560,0.000000,0.000000,0.000000,2008.000000,341.000000,68.000000,2.000000,1.000000,0.280000,2.000000,1.000000,4.000000,1.000000
50%,2.000000,97.000000,40.722280,-73.954430,0.000000,1.000000,0.000000,2012.000000,624.000000,125.000000,3.000000,7.000000,0.740000,3.000000,1.000000,96.000000,1.000000
75%,2.000000,178.000000,40.762770,-73.932320,1.000000,2.000000,2.000000,2017.000000,912.000000,182.000000,5.000000,30.000000,1.710000,4.000000,2.000000,268.000000,1.000000
max,4.000000,223.000000,40.916970,-73.705220,1.000000,2.000000,3.000000,2022.000000,1200.000000,240.000000,365.000000,1024.000000,90.000000,5.000000,332.000000,365.000000,1.000000
